In [72]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [73]:
#Carregando o dataset
df = pd.read_excel("../dataset/dry_bean_dataset.xls")

In [74]:
#Dividindo a base em treino e teste
X = df.drop("Class", axis=1)
y = df["Class"]

#Transformando a variável cartegorica em binária
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=123)

#Normalizando os dados
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

In [75]:
#Aplicando o modelo KNN

#Definindo o valor de vizinhos
classifier = KNeighborsClassifier(n_neighbors=5)

#Treinar o modelo, com os dados de treinamento
classifier.fit(X_train, y_train)

#Prever os valores de y com dos dados de X_test
y_pred = classifier.predict(X_test)

# Imprimindo a matriz confusa
print("Matriz Confusa: ")
print(confusion_matrix(y_test, y_pred), "\n")  

# Imprimindo o relatório de classificação
print("Relatório de classificação: \n", classification_report(y_test, y_pred))  

# Imprimindo o quão acurado foi o modelo
print('Acurácia do modelo: ' , accuracy_score(y_test, y_pred))

Matriz Confusa: 
[[ 387    0   30    0    2    4   13]
 [   0  171    0    0    0    0    0]
 [  13    0  504    0    6    2    7]
 [   0    0    0 1051    1   11   81]
 [   0    0   19    3  614    0   18]
 [   5    0    0   23    1  625   20]
 [   2    0    0   93   10   10  766]] 

Relatório de classificação: 
               precision    recall  f1-score   support

           0       0.95      0.89      0.92       436
           1       1.00      1.00      1.00       171
           2       0.91      0.95      0.93       532
           3       0.90      0.92      0.91      1144
           4       0.97      0.94      0.95       654
           5       0.96      0.93      0.94       674
           6       0.85      0.87      0.86       881

    accuracy                           0.92      4492
   macro avg       0.93      0.93      0.93      4492
weighted avg       0.92      0.92      0.92      4492

Acurácia do modelo:  0.9167408726625111


In [76]:
#Aplicando a Árvore de decisão

#Instanciando o modelo
model = DecisionTreeClassifier()

# Treinar o modelo
model.fit(X_train, y_train)

# Aplicar o modelo ao treinamento e ao teste
predicted_test_y = model.predict(X_test)

predicted_train_y = model.predict(X_train)

In [77]:
#Imprimindo a acurácia do modelo
accuracy = accuracy_score(y_test, predicted_test_y) * 100
print ("A acurácia foi de {:.2f}%.".format(accuracy))

A acurácia foi de 89.94%.


In [78]:
#Aplicando a Random Forest